In [1]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split

# Load dataset
data_df = pd.read_csv("train.csv")

# Split dataset into training, validation, and testing sets
train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=42, stratify=data_df['has_under_extrusion'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['has_under_extrusion'])

In [2]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# create an instance of the ImageDataGenerator class
data_gen = ImageDataGenerator(
    preprocessing_function=lambda image: cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), IMG_SIZE).astype(np.float32) / 255.0
)

# specify the path to the top-level directory containing all the subdirectories
data_dir = 'Images'

# use the flow_from_directory function to generate batches of preprocessed images
image_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# loop over the batches of images
for i, (batch_images, batch_labels) in enumerate(image_generator):
    # perform some operation on the batch of images here
    # for example, feed the batch to a neural network model
    
    # check if we have processed all the batches
    if i == len(image_generator) - 1:
        break


Found 106339 images belonging to 7 classes.


In [1]:
train_df["has_under_extrusion"] = train_df["has_under_extrusion"].astype(str)
val_df["has_under_extrusion"] = val_df["has_under_extrusion"].astype(str)
test_df["has_under_extrusion"] = test_df["has_under_extrusion"].astype(str)

NameError: name 'train_df' is not defined

In [10]:
train_df

,img_path,printer_id,print_id,has_under_extrusion
117,101/1678589738/1678589962.082321.jpg,101,1678589738,1
27824,102/1678766920/1678767290.65399.jpg,102,1678766920,1
25771,102/1678816535/1678817797.548483.jpg,102,1678816535,1
26666,102/1678816535/1678818166.220234.jpg,102,1678816535,1
24899,102/1678816535/1678817437.772464.jpg,102,1678816535,1
...,...,...,...,...
37195,102/1678739903/1678740959.434142.jpg,102,1678739903,0
16285,101/1678341667/1678342560.798175.jpg,101,1678341667,0
11118,101/1678333984/1678334644.186267.jpg,101,1678333984,0
74511,022/1672773342/1672773636.389223.jpg,22,1672773342,0


In [11]:
data_dir = 'Image'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = data_gen.flow_from_dataframe(
    dataframe=train_df,
    directory=data_dir,
    x_col="img_path",
    y_col="has_under_extrusion",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

valid_generator = data_gen.flow_from_dataframe(
    dataframe=val_df,
    directory=data_dir,
    x_col="img_path",
    y_col="has_under_extrusion",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,
    
)
test_generator = data_gen.flow_from_dataframe(
    dataframe=test_df,
    directory=data_dir,
    x_col="img_path",
    y_col="has_under_extrusion",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)






Found 51878 validated image filenames belonging to 2 classes.
Found 12970 validated image filenames belonging to 2 classes.
Found 16212 validated image filenames belonging to 2 classes.


In [12]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model


# Load the pre-trained VGG-16 model
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in vgg.layers:
    layer.trainable = False

# Add a few layers on top of the pre-trained model for your specific task
x = Flatten()(vgg.output)
x = Dense(64, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

# Create the final model
model = Model(inputs=vgg.input, outputs=outputs)






In [15]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.00001), metrics=['accuracy'])

# Define batch size and number of epochs
batch_size = 32
epochs = 5

# Train the model and save after every epoch
history = model.fit(train_generator, epochs=epochs, validation_data=valid_generator, steps_per_epoch=train_generator.samples//batch_size, validation_steps=valid_generator.samples//batch_size, callbacks=[ModelCheckpoint('model.h5', save_best_only=True)])

# Evaluate the model on the test set


Epoch 1/5


2023-04-12 19:33:45.096462: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1621/1621 [==============================] - 554s 341ms/step - loss: 0.1234 - accuracy: 0.9474 - val_loss: 0.0340 - val_accuracy: 0.9870
Epoch 2/5
1621/1621 [==============================] - 557s 343ms/step - loss: 0.1203 - accuracy: 0.9472 - val_loss: 0.0340 - val_accuracy: 0.9870
Epoch 3/5
1621/1621 [==============================] - 554s 342ms/step - loss: 0.1202 - accuracy: 0.9480 - val_loss: 0.0325 - val_accuracy: 0.9879
Epoch 4/5
1621/1621 [==============================] - 554s 341ms/step - loss: 0.1175 - accuracy: 0.9502 - val_loss: 0.0328 - val_accuracy: 0.9877
Epoch 5/5
1621/1621 [==============================] - 552s 341ms/step - loss: 0.1178 - accuracy: 0.9491 - val_loss: 0.0317 - val_accuracy: 0.9885


In [16]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

2023-04-12 21:16:57.631115: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


507/507 [==============================] - 138s 272ms/step - loss: 0.0296 - accuracy: 0.9898
Test loss: 0.029585517942905426
Test accuracy: 0.98976069688797


In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(test_generator)
y_pred = np.round(y_pred).astype(int)

# Get the ground truth labels from the test generator
y_true = test_generator.classes

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print the evaluation metrics
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 score:', f1)

2023-04-12 21:30:21.354231: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


507/507 [==============================] - 139s 273ms/step
Accuracy: 0.9897606711078214
Precision: 0.9940949352714059
Recall: 0.9871447902571042
F1 score: 0.9906076722869752


In [19]:

predictions = model.predict(test_generator)

# Convert predictions to binary values (0 or 1)
binary_predictions = np.round(predictions).flatten()






2023-04-12 21:39:44.269374: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


507/507 [==============================] - 139s 273ms/step


In [20]:
from sklearn.metrics import classification_report
class_names = ["No Under Extrusion", "Under Extrusion"]
print(classification_report(test_generator.labels, binary_predictions, target_names=class_names))

                    precision    recall  f1-score   support

No Under Extrusion       0.98      0.99      0.99      7344
   Under Extrusion       0.99      0.99      0.99      8868

          accuracy                           0.99     16212
         macro avg       0.99      0.99      0.99     16212
      weighted avg       0.99      0.99      0.99     16212



In [21]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
test_df1=pd.read_csv("test.csv")
# create an instance of the ImageDataGenerator clas


# specify the path to the top-level directory containing all the subdirectories
data_dir = 'Image'

test_generator1 = data_gen.flow_from_dataframe(
    dataframe=test_df1,
    directory=data_dir,
    x_col="img_path",
    y_col=None,
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)



Found 25279 validated image filenames.


In [22]:
test_predictions = model.predict(test_generator1)
binary_test_predictions = np.round(test_predictions).flatten()


2023-04-12 22:20:30.246673: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


790/790 [==============================] - 239s 303ms/step


In [23]:
submission_df = pd.DataFrame({'img_path': test_df1['img_path'], 'has_under_extrusion': binary_test_predictions})
submission_df.to_csv('submission_best.csv', index=False)

In [24]:
df1=pd.read_csv("submission_best.csv")
df1

,img_path,has_under_extrusion
0,101/1678578332/1678578538.704966.jpg,1.0
1,101/1678578332/1678578539.108019.jpg,1.0
2,101/1678578332/1678578539.512872.jpg,1.0
3,101/1678578332/1678578539.916711.jpg,1.0
4,101/1678578332/1678578540.329089.jpg,1.0
...,...,...
25274,022/1672795514/1672796229.177132.jpg,1.0
25275,022/1672795514/1672796230.177629.jpg,1.0
25276,022/1672795514/1672796231.187994.jpg,1.0
25277,022/1672795514/1672796232.19111.jpg,1.0


In [25]:
df1['has_under_extrusion'] = df1['has_under_extrusion'].astype(int)

In [26]:
df1.to_csv('submission3.csv', index=False)